In [1]:
%pip install pandas openpyxl ecopy comb scikit-bio
import pandas as pd
import os
from skbio.diversity import alpha


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Paths
HERE = os. getcwd()
DATA_DIRECTORY = os.path.abspath(os.path.join(HERE, os.pardir,  os.pardir, 'data'))
RESULT_DIRECTORY = os.path.abspath(os.path.join(HERE, os.pardir, os.pardir, 'results', 'tables','complexity'))
MICROBIOME_RAW_ABUNDANCE_PATH = os.path.join(DATA_DIRECTORY, 'processed', 'new_GTDB_cecum_species_raw_abundance_filterd_low.xlsx')
METADATA_PATH = os.path.join(DATA_DIRECTORY, 'metadata', 'metadata.xlsx')


In [3]:
# Load microbiome raw counts and metadata
raw_species_table = pd.read_excel(MICROBIOME_RAW_ABUNDANCE_PATH, index_col=0)
metadata = pd.read_excel(METADATA_PATH, header=0, index_col=0)
metadata

,SAMPLED_BIRD_ID,GUT_SECTION,TYPE,AGE,TREATMENT
sample-id,,,,,
cD14A1,D14A1,Cecum,Digesta,14,CTR
cD14A2,D14A2,Cecum,Digesta,14,CTR
cD14A3,D14A3,Cecum,Digesta,14,CTR
cD14A4,D14A4,Cecum,Digesta,14,CTR
cD14B1,D14B1,Cecum,Digesta,14,PFA
...,...,...,...,...,...
cM3B4,D3B4,Cecum,Mucosa,3,PFA
cM3C1,D3C1,Cecum,Mucosa,3,AGP
cM3C2,D3C2,Cecum,Mucosa,3,AGP


In [4]:
raw_species_table

,Clostridium_J sp000821305,Azonexus hortensis,Alistipes senegalensis,Alistipes_A ihumii,UBA6398 sp003150315,Parabacteroides merdae,Alistipes shahii,Alistipes communis,Alistipes dispar,Alistipes sp900548155,...,Lawsonibacter sp900545895,Erysipelatoclostridium spiroforme,Streptococcus alactolyticus,Merdisoma faecalis,Dietzia natronolimnaea_A,Mediterraneibacter faecipullorum,Escherichia coli,Caccovicinus sp017889125,Lawsonibacter pullicola,Dietzia cercidiphylli
sample-id,,,,,,,,,,,,,,,,,,,,,
cD14A1,0,0,0,0,0,0,0,0,0,0,...,21074,55749,4676,35680,205,50894,1072,27979,22570,1194
cD14A2,0,0,0,0,0,0,0,0,0,0,...,38494,7927,1749,42898,208,9480,280,10382,74969,1020
cD14A3,0,0,0,0,0,0,0,0,0,0,...,24181,18506,445,3185,221,40370,1187,149,29666,1305
cD14A4,0,0,0,0,0,0,0,0,0,0,...,35672,3052,216,11070,192,10454,449,3007,32420,1015
cD14B1,0,0,0,0,0,0,0,0,0,0,...,34765,22599,3946,32405,225,15355,3984,1217,29598,1320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cM3B4,0,0,0,0,0,0,0,0,0,0,...,0,173,726,0,187,0,936,0,0,1187
cM3C1,0,0,0,0,0,0,0,0,0,0,...,0,0,95,0,249,730,214,0,0,1070
cM3C2,0,24,0,0,0,0,0,0,0,0,...,0,1163,267,0,180,1266,1456,0,180,900


In [5]:
def calculate_alpha_diversity(metric_func, metric_name, measure_type, age):
    # Filter metadata for the specified age and sample type
    metadata_filtered = metadata[
        (metadata['TYPE'] == measure_type) &
        (metadata['AGE'] == age)
    ]

    # Match sample order in the count table and metadata table
    raw_counts = raw_species_table.loc[metadata_filtered.index]


    # Calculate the diversity
    raw_counts.apply(metric_func, axis=1)
   
    # Construct the output file name
    age=str(age)
    file_name = f"microbiome_{measure_type}_{age}_metadata_{metric_name.lower().replace(' ', '_')}.xlsx"

    # Save the diversity calculation
    output_path = os.path.join(RESULT_DIRECTORY, file_name)
    metadata_filtered[metric_name] = raw_counts.apply(metric_func, axis=1)
    metadata_filtered.to_excel(output_path, index_label='SAMPLE_ID')

# Define a dictionary of functions for different metrics
metrics = {
    'Shannon_Diversity': alpha.shannon,
    'Chao1_Richness': alpha.chao1,
    'Pielou_Evenness': alpha.pielou_e
}

In [6]:
measure_types = ['Digesta', 'Mucosa']
ages = [3, 14, 21, 35]


for measure_type in measure_types:
    for age in ages:
        for metric_name, metric_func in metrics.items():
            calculate_alpha_diversity(metric_func, metric_name, measure_type, age)


/tmp/ipykernel_326973/795803723.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_filtered[metric_name] = raw_counts.apply(metric_func, axis=1)
/tmp/ipykernel_326973/795803723.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_filtered[metric_name] = raw_counts.apply(metric_func, axis=1)
/tmp/ipykernel_326973/795803723.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave